> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 2.0`: reinforcement learning exploration demo
This illustrates a use case where we aim to achieve an exploration behavior by generating as many as possible diverse solutions by using a predictive model as the main component.

NOTE: The generated solutions might not be entirely reliable since they could be outside of the applicability domain of the predictive model. Predictive models could score highly compounds that are outside of the applicability domain but this score would be likely inaccurate. This mode would be more reliable if we aslo include `matching_substructure` component with a list of desired core structural patterns/scaffolds. Alternatively this mode can be quite successful in combination with docking or pharmacophore similarity. Such examples will be provided with the next releases. 


## 1. Set up the paths
_Please update the following code block such that it reflects your system's installation and execute it._

In [2]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/Projects/Publications/2020/2020-04_REINVENT_2.0/Reinvent")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent_shared.v2.1")
output_dir = os.path.expanduser("~/Desktop/REINVENT_RL_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

In [14]:
reinvent_dir = os.path.expanduser("~/PycharmProjects/github_repositories/reinvent/")

## 2. Setting up the configuration 
In the cells below we will build a nested dictionary object that will be eventually converted to JSON file which in turn will be consumed by `REINVENT`. 
You can find this file in your `output_dir` location.

### A) Declare the run type

In [15]:
# initialize the dictionary
configuration = {
    "version": 2,                          # we are going to use REINVENT's newest release
    "run_type": "reinforcement_learning"   # other run types: "sampling", "validation",
                                           #                  "transfer_learning",
                                           #                  "scoring" and "create_model"
}

### B) Sort out the logging details
This includes `resultdir` path where the results will be produced.

Also: `REINVENT` can send custom log messages to a remote location. We have retained this capability in the code. if the `recipient` value differs from `"local"` `REINVENT` will attempt to POST the data to the specified `recipient`. 

In [16]:
# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "http://0.0.0.1",          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                  # either to local logging or use a remote REST-interface
    "logging_frequency": 10,               # log every x-th steps
    "logging_path": os.path.join(output_dir, "progress.log"), # load this folder in tensorboard
    "resultdir": os.path.join(output_dir, "results"),         # will hold the compounds (SMILES) and summaries
    "job_name": "Reinforcement learning demo",                # set an arbitrary job name for identification
    "job_id": "demo"                       # only relevant if "recipient" is set to a specific REST endpoint
}

Create `"parameters"` field

In [17]:
# add the "parameters" block
configuration["parameters"] = {}

### C) Set Diversity Filter
During each step of Reinforcement Learning the compounds scored above `minscore` threshold are kept in memory. The scored smiles are written out to a file in the results folder `scaffold_memory.csv`. In the example here we are not using any filter by setting it to `"IdenticalMurckoScaffold"`. This will help to explore the chemical space since using the diversity filters will stimulate generation of more diverse solutions. The maximum average value of the scoring fuinction will be lower in exploration mode because the Agent is encouraged to search for diverse scaffolds rather than to only optimize the ones that are being found so far. The number of generated compounds should be higher in comparison to the exploitation scenario since the diversity is encouraged. 

In [18]:
# add a "diversity_filter"
configuration["parameters"]["diversity_filter"] =  {
    "name": "IdenticalMurckoScaffold",     # other options are: "IdenticalTopologicalScaffold", 
                                           # "IdenticalMurckoScaffold" and "ScaffoldSimilarity"
                                           # -> use "NoFilter" to disable this feature
    "nbmax": 25,                           # the bin size; penalization will start once this is exceeded
    "minscore": 0.4,                       # the minimum total score to be considered for binning
    "minsimilarity": 0.4                   # the minimum similarity to be placed into the same bin
}

### D) Set Inception
* `smiles` provide here a list of smiles to be incepted 
* `memory_size` the number of smiles allowed in the inception memory
* `sample_size` the number of smiles that can be sampled at each reinforcement learning step from inception memory

In [19]:
# prepare the inception (we do not use it in this example, so "smiles" is an empty list)
configuration["parameters"]["inception"] = {
    "smiles": [],                          # fill in a list of SMILES here that can be used (or leave empty)
    "memory_size": 100,                    # sets how many molecules are to be remembered
    "sample_size": 10                      # how many are to be sampled each epoch from the memory
}

### E) Set the general Reinforcement Learning parameters
* `n_steps` is the amount of Reinforcement Learning steps to perform. Best start with 1000 steps and see if thats enough.
* `agent` is the generative model that undergoes transformation during the Reinforcement Learning run.

We reccomend keeping the other parameters to their default values.

In [20]:
# set all "reinforcement learning"-specific run parameters
configuration["parameters"]["reinforcement_learning"] = {
    "prior": os.path.join(ipynb_path, "models/augmented.prior"), # path to the pre-trained model
    "agent": os.path.join(ipynb_path, "models/augmented.prior"), # path to the pre-trained model
    "n_steps": 1000,                        # the number of epochs (steps) to be performed; often 1000
    "sigma": 128,                          # used to calculate the "augmented likelihood", see publication
    "learning_rate": 0.0001,               # sets how strongly the agent is influenced by each epoch
    "batch_size": 128,                     # specifies how many molecules are generated per epoch
    "reset": 0,                            # if not '0', the reset the agent if threshold reached to get
                                           # more diverse solutions
    "reset_score_cutoff": 0.5,             # if resetting is enabled, this is the threshold
    "margin_threshold": 50                 # specify the (positive) margin between agent and prior
}

### F) Define the scoring function
We will use a `custom_product` type. The component types included are:
* `predictive_property` which is the target activity to _Aurora_ kinase represented by the predictive `regression` model. Note that we set the weight of this component to be the highest.
* `qed_score` is the implementation of QED in RDKit. It biases the egenration of  molecules towars more "drug-like" space. Depending on the study case can have beneficial or detrimental effect.
* `custom_alerts` the `"smiles"` field  also can work with SMILES or SMARTS

Note: The model used in this example is a regression model


In [21]:
# prepare the scoring function definition and add at the end
scoring_function = {
    "name": "custom_product",              # this is our default one (alternative: "custom_sum")
    "parallel": False,                     # sets whether components are to be executed
                                           # in parallel; note, that python uses "False" / "True"
                                           # but the JSON "false" / "true"

    # the "parameters" list holds the individual components
    "parameters": [

    # add component: an activity model
    {
        "component_type": "predictive_property", # this is a scikit-learn model, returning
                                                 # activity values
        "name": "Aurora kinase",        # arbitrary name for the component
        "weight": 6,                       # the weight ("importance") of the component (default: 1)
        "model_path": os.path.join(ipynb_path, "models/Aurora_model.pkl"),   # absolute model path
        "smiles": [],                            # list of SMILES (not required for this component)
        "specific_parameters": {
            "transformation_type": "sigmoid",  # see description above
            "high": 9,                         # parameter for sigmoid transformation
            "low": 4,                          # parameter for sigmoid transformation
            "k": 0.25,                         # parameter for sigmoid transformation
            "scikit": "regression",            # model can be "regression" or "classification"
            "transformation": True,            # enable the transformation
            "descriptor_type": "ecfp_counts",  # sets the input descriptor for this model
            "size": 2048,                      # parameter of descriptor type
            "radius": 3,                       # parameter of descriptor type
            "use_counts": True,                # parameter of descriptor type
            "use_features": True               # parameter of descriptor type
        }
    },

    # add component: QED
    {
        "component_type": "qed_score", # this is the QED score as implemented in RDKit
        "name": "QED",        # arbitrary name for the component
        "weight": 2,            # the weight ("importance") of the component (default: 1)
        "model_path": None,
        "smiles":  None                         
    },

    # add component: enforce to NOT match a given substructure
    {
        "component_type": "custom_alerts",
        "name": "Custom alerts",               # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "model_path": None,                    # not required; note, this is "null" in JSON
        "smiles": [                            # specify the substructures (as list) to penalize
            "[*;r8]",
            "[*;r9]",
            "[*;r10]",
            "[*;r11]",
            "[*;r12]",
            "[*;r13]",
            "[*;r14]",
            "[*;r15]",
            "[*;r16]",
            "[*;r17]",
            "[#8][#8]",
            "[#6;+]",
            "[#16][#16]",
            "[#7;!n][S;!$(S(=O)=O)]",
            "[#7;!n][#7;!n]",
            "C#C",
            "C(=[O,S])[O,S]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
        ],
        "specific_parameters": None            # not required; note, this is "null" in JSON
    }]
}
configuration["parameters"]["scoring_function"] = scoring_function

#### NOTE:  Getting the selectivity score component to reach satisfactory levels is non-trivial and might take considerably higher number of steps

## 3. Write out the configuration

We now have successfully filled the dictionary and will write it out as a `JSON` file in the output directory. Please have a look at the file before proceeding in order to see how the paths have been inserted where required and the `dict` -> `JSON` translations (e.g. `True` to `true`) have taken place.

In [22]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "RL_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

## 4. Run `REINVENT`
Now it is time to execute `REINVENT` locally. Note, that depending on the number of epochs (steps) and the execution time of the scoring function components, this might take a while. 

The command-line execution looks like this:
```
# activate envionment
conda activate reinvent_shared.v2.1

# execute REINVENT
python <your_path>/input.py <config>.json
```

In [23]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!python {reinvent_dir}/input.py {configuration_JSON_path}

In [13]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

# prepare the output to be parsed
list_epochs = re.findall(r'INFO.*?local', captured_err_stream.stdout, re.DOTALL)
data = [epoch for idx, epoch in enumerate(list_epochs) if idx in [1, 75, 124]]
data = ["\n".join(element.splitlines()[:-1]) for element in data]

Below you see the print-out of the first, one from the middle and the last epoch, respectively. Note, that the fraction of valid `SMILES` is high right from the start (because we use a pre-trained prior). You can see the partial scores for each component for the first couple of compounds, but the most important information is the average score. You can clearly see how it increases over time.

In [12]:
for element in data:
    print(element)

INFO     
 Step 0   Fraction valid SMILES: 99.2   Score: 0.0614   Time elapsed: 0   Time left: 0.0
  Agent     Prior     Target     Score     SMILES
-25.73    -25.73    -16.42      0.07      c1cc(OCc2cc(C(N(CC3CCOCC3)CC)=O)no2)ccc1SC
-22.98    -22.98    -12.92      0.08      C1CCCN1c1nc(N2CCOCC2)nc(-c2ccccc2)c1
-22.20    -22.20    -22.20      0.00      c1cccc2n(CC)c(SCC(=O)NCc3ccc4c(c3)OCO4)nc12
-32.45    -32.45    -13.65      0.15      c1c(S(=O)(=O)N2CC(c3ccccc3)N(Cc3ccccc3)C(=O)C2)cc(F)cc1F
-22.47    -22.47     -7.58      0.12      c1(CCc2oc(=O)cc(OC)c2CO)ccccc1
-20.22    -20.22    -11.42      0.07      C1(C)CN(CCC(Nc2c(OC)cccc2)=O)CC(C)O1
-36.57    -36.57    -36.57      0.00      OC(CCC(=O)NC(c1nc(-c2cccc(C(=O)NC(C)(C)C)c2)c[nH]1)Cc1ccccc1)=O
-31.60    -31.60    -21.74      0.08      O=S(C1C2CCN(C1)CC2)(c1ccc(Oc2cccnc2C)cc1)=O
-26.47    -26.47    -15.55      0.09      c12c([nH]c3c(c1=NCCCNC(C)C)cccc3)CC(C)CC2
-41.70    -41.70    -41.70      0.00      C(C1(C)NC(C(NC(C=O)CCCN=C(N)N)=O

## 5. Analyse the results
In order to analyze the run in a more intuitive way, we can use `tensorboard`:

```
# go to the root folder of the output
cd <your_path>/REINVENT_RL_demo

# make sure, you have activated the proper environment
conda activate reinvent_shared.v2.1

# start tensorboard
tensorboard --logdir progress.log
```

Then copy the link provided to a browser window, e.g. "http://workstation.url.com:6006/". The following figures are exmaple plots - remember, that there is always some randomness involved. In `tensorboard` you can monitor the individual scoring function components. 

The score for predicted Aurora Kinase activity.

![](img/explore_aurora_kinase.png)

The average score over time.

![](img/explore_avg_score.png)

It might also be informative to look at the results from the prior (dark blue), the agent (blue) and the augmented likelihood (purple) over time.

![](img/explore_nll_plot.png)

And last but not least, there is a "Images" tab available that lets you browse through the compounds generated in an easy way. In the molecules, the substructure matches that were defined to be required are highlighted in red (if present). Also, the total scores are given per molecule.

![](img/molecules.png)

The results folder will hold four different files: the agent (pickled), the input JSON (just for reference purposes), the memory (highest scoring compounds in `CSV` format) and the scaffold memory (in `CSV` format).

In [13]:
!head -n 15 {output_dir}/results/memory.csv

,smiles,score,likelihood
65,C(CCCn1cc(C(C)(C)C)c2c(C(C)C)cc(C(C)C)cc2c1=O)C(=O)N=c1nc[nH][nH]1,0.3286117,-50.641468
70,C1C(N(CCC)CCC)Cc2cccc3[nH]c(=O)n(c32)C1,0.32649106,-18.146914
26,O1c2c(nc(OC)cc2)C(C(NCCCN(C)C)=O)(Cc2ccccc2)c2ccccc21,0.32437962,-35.405247
60,c1c(C(CNCCc2ccc(NS(=O)(c3ccc(-c4oc(Cc5c[nH]c(=N)s5)cc4)nc3)=O)cc2)O)c[nH]c(=N)c1,0.32314676,-38.32259
99,c1c2c(cc(Cl)c1Cl)C(CC(=O)c1cnn(C)c1)(O)C(=O)N2,0.31027606,-27.762121
11,c1c(O)c(C(Cc2ccc(Cl)cc2)=O)cc(O)c1Oc2c(O)cc(O)cc2CCC(O)c1cc(O)c(OC)cc1,0.30576745,-52.903526
32,c1(C(NC(Cc2ccccc2)C(C(NCCN2CCOCC2)=O)=O)=O)cc(C(=O)NS(Cc2ccccc2)(=O)=O)c(NCCC)s1,0.30178678,-43.933296
1,c1c(C(C)C)ccc(NC(c2cc3c(cc2)[nH]c2c(C(N)=O)ccc(O)c23)=O)c1,0.30052438,-31.108843
108,c1(C(C(F)(F)F)(F)F)cc(Cn2c3cccc(NC(c4n5ccc(OCCN6CCN(C)CC6)cc5nc4)=O)c3c(CC)n2)ccc1,0.29700187,-34.311478
118,c1ccc(C(COc2ccc3c(occ(Oc4ccccc4)c3=O)c2)(O)C(N2CCCCC2)C)cc1F,0.29602197,-45.389744
109,C1CN(CC(CNC(c2ccc3n(c(=O)cc(C)n3)c2)=O)O)CCC1Cc1ccccc1,0.29525602,-29.0487
19,